<a href="https://colab.research.google.com/github/yektacc/epet24/blob/master/Rubik_Ai_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pycuber
!pip install stable-baselines3
!pip install gym
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [9]:
!pip install shimmy

In [24]:
import gym
from gym import spaces
import numpy as np
from pycuber import *
from stable_baselines3 import PPO
import random

class RubikEnv(gym.Env):
    def __init__(self):
        super(RubikEnv, self).__init__()
        self.cube = Cube()
        self.action_space = spaces.Discrete(18)
        self.observation_space = spaces.Box(low=0, high=5, shape=(54,), dtype=np.uint8)

    def reset(self):
        self.cube = Cube()
        moves = ['U', "U'", 'R', "R'", 'F', "F'", 'D', "D'", 'L', "L'", 'B', "B'", 'U2', 'R2', 'F2', 'D2', 'L2', 'B2']
        scramble_moves = random.choices(moves, k=20)
        for move in scramble_moves:
            self.cube(move)
        return self._get_observation()

    def _get_observation(self):
        state = np.zeros(54, dtype=np.uint8)
        faces = ['U', 'R', 'F', 'D', 'L', 'B']
        color_map = {'W': 0, 'R': 1, 'G': 2, 'O': 3, 'Y': 4, 'B': 5} # ایجاد دیکشنری رنگ ها
        for i, face_name in enumerate(faces):
            # Accessing the face using a string key
            face = self.cube.get_face(face_name)
            for j in range(9):
                # Accessing element using row and column index separately
                row = j % 3
                col = j // 3
                # Accessing element using row and column index separately
                color = str(face[row][col]).upper()  # Access element using row, then col
                state[i * 9 + j] = color_map.get(color, 0) # استفاده از دیکشنری برای نگاشت رنگ به عدد
        return state

    def step(self, action):
        moves = ['U', "U'", 'R', "R'", 'F', "F'", 'D', "D'", 'L', "L'", 'B', "B'", 'U2', 'R2', 'F2', 'D2', 'L2', 'B2']
        self.cube(moves[action])
        observation = self._get_observation()
        reward = self._get_reward()
        # Instead of is_solved, we need to compare the cube with a solved cube.
        done = self.cube == Cube()
        return observation, reward, done, {}

    def _get_reward(self):
        # Instead of is_solved, we need to compare the cube with a solved cube.
        if self.cube == Cube():
            return 100
        else:
            return -1

# ایجاد محیط مکعب روبیک
env = RubikEnv()

# ایجاد و آموزش مدل PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# ذخیره مدل آموزش‌دیده
model.save("rubik_ppo")

print("فایل rubik_ppo.zip با موفقیت ایجاد شد.")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 197  |
|    iterations      | 1    |
|    time_elapsed    | 10   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 185          |
|    iterations           | 2            |
|    time_elapsed         | 22           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 7.659572e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.89        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 13.9         |
|    n_updates            | 10           |
|    policy_gradient_loss | 0.00264      |
|    val

In [25]:
import gym
from gym import spaces
import numpy as np
from pycuber import *
from stable_baselines3 import PPO
import random
import csv

class RubikEnv(gym.Env):
    def __init__(self):
        super(RubikEnv, self).__init__()
        self.cube = Cube()
        self.action_space = spaces.Discrete(18)
        self.observation_space = spaces.Box(low=0, high=5, shape=(54,), dtype=np.uint8)

    def reset(self):
        self.cube = Cube()
        moves = ['U', "U'", 'R', "R'", 'F', "F'", 'D', "D'", 'L', "L'", 'B', "B'", 'U2', 'R2', 'F2', 'D2', 'L2', 'B2']
        scramble_moves = random.choices(moves, k=20)
        for move in scramble_moves:
            self.cube(move)
        return self._get_observation()

    def _get_observation(self):
        state = np.zeros(54, dtype=np.uint8)
        faces = ['U', 'R', 'F', 'D', 'L', 'B']
        color_map = {'W': 0, 'R': 1, 'G': 2, 'O': 3, 'Y': 4, 'B': 5}
        for i, face_name in enumerate(faces):
            face = self.cube.get_face(face_name) # Access the face using get_face
            for j in range(9):
                row = j % 3
                col = j // 3
                color = str(face[row][col]).upper()  # Access element using row, then col
                state[i * 9 + j] = color_map.get(color, 0)
        return state

    def step(self, action):
        moves = ['U', "U'", 'R', "R'", 'F', "F'", 'D', "D'", 'L', "L'", 'B', "B'", 'U2', 'R2', 'F2', 'D2', 'L2', 'B2']
        self.cube(moves[action])
        observation = self._get_observation()
        reward = self._get_reward()
        # Instead of is_solved(), compare the cube with a solved Cube object
        done = self.cube == Cube()
        return observation, reward, done, {}

    def _get_reward(self):
        # Instead of is_solved(), compare the cube with a solved Cube object
        if self.cube == Cube():
            return 100
        else:
            return -1


# آموزش مدل
env = RubikEnv()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("rubik_ppo")

# حل مکعب روبیک تصادفی
model = PPO.load("rubik_ppo")
env = RubikEnv()
obs = env.reset()
done = False
move_count = 0
moves_list = ['U', "U'", 'R', "R'", 'F', "F'", 'D', "D'", 'L', "L'", 'B', "B'", 'U2', 'R2', 'F2', 'D2', 'L2', 'B2']

with open('rubik_solution.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Move', 'Cube State'])
    writer.writerow(['Initial State', str(env.cube)])

    print("وضعیت اولیه مکعب روبیک:")
    print(env.cube)

    while not done:
        action, _states = model.predict(obs)
        obs, rewards, done, info = env.step(action)
        move_name = moves_list[action]
        print(f"حرکت {move_count + 1}: {move_name}")
        print(env.cube)
        writer.writerow([move_name, str(env.cube)])
        move_count += 1

# Instead of is_solved(), compare the cube with a solved Cube object
if env.cube == Cube():
    print(f"مکعب روبیک در {move_count} حرکت حل شد!")
else:
    print("مدل نتوانست مکعب روبیک را حل کند.")

Streaming output truncated to the last 5000 lines.
[r][r][r][y][g][b][y][o][w][b][b][b]
[g][o][w][o][r][g][y][y][b][r][g][y]
         [b][w][o]
         [g][w][o]
         [r][w][w]

حرکت 46104: R'
         [r][y][r]
         [g][y][b]
         [o][o][b]
[g][r][g][w][w][r][y][w][b][w][g][w]
[r][r][r][y][g][b][o][o][y][o][b][b]
[g][o][w][o][r][y][b][y][y][o][g][y]
         [b][w][o]
         [g][w][b]
         [r][w][g]

حرکت 46105: U2
         [b][o][o]
         [b][y][g]
         [r][y][r]
[y][w][b][w][g][w][g][r][g][w][w][r]
[r][r][r][y][g][b][o][o][y][o][b][b]
[g][o][w][o][r][y][b][y][y][o][g][y]
         [b][w][o]
         [g][w][b]
         [r][w][g]

حرکت 46106: U2
         [r][y][r]
         [g][y][b]
         [o][o][b]
[g][r][g][w][w][r][y][w][b][w][g][w]
[r][r][r][y][g][b][o][o][y][o][b][b]
[g][o][w][o][r][y][b][y][y][o][g][y]
         [b][w][o]
         [g][w][b]
         [r][w][g]

حرکت 46107: L'
         [w][y][r]
         [y][y][b]
         [o][o][b]
[g][r][w][b][w][r][y][

KeyboardInterrupt: 